In [10]:
# Carica il file csv e converto colonna dei prezzi in int
import pandas as pd
import re

df=pd.read_csv("cosenza_dati_booking.csv")
df['prezzo'] = df['prezzo'].apply(lambda x: int(re.search(r'\d+', x).group()))
df['prezzo_iniziale'] = df['prezzo_iniziale'].apply(lambda x: int(re.search(r'\d+', x).group()))
df.columns

Index(['nome_hotel', 'prezzo_iniziale', 'prezzo', 'stanza', 'città', 'data',
       'punteggio', 'numero_recensioni', 'distanza_centro', 'genius',
       'offerte', 'colazione_inclusa', 'senza_pagamento_anticipato',
       'cancellazione_gratuita', 'os', 'username', 'user_agent_type',
       'TIMESTAMP'],
      dtype='object')

In [11]:
# Crea delle colonne con valori True/False in base alle offerte
df['booking_paga'] = df['offerte'].str.contains('Booking.com paga')
df['offerta_tempo_limitato'] = df['offerte'].str.contains('tempo limitato')
df['offerta_solo_mobile'] = df['offerte'].str.contains('mobile')
df['offerta_super_segreta'] = df['offerte'].str.contains('SuperSegreta')
df['offerta_inizio_2024'] = df['offerte'].str.contains('Offerta Inizio 2024')
# Inserisce False nei campi nulli
df[['booking_paga','offerta_tempo_limitato','offerta_solo_mobile','offerta_super_segreta','offerta_inizio_2024']] = df[['booking_paga','offerta_tempo_limitato','offerta_solo_mobile','offerta_super_segreta','offerta_inizio_2024']].fillna(False)

In [12]:
# Visualizzo OS in base a username
df_user = df[df['username']=='pantilaura56']
lst = df_user['TIMESTAMP'].unique()
os = []
for i in lst:
    df3 = df_user[df_user['TIMESTAMP']==i]
    os.append(df3['os'].unique()[0])
os

['Linux']

In [13]:
# Visualizza lista ricerche per orario
df['TIMESTAMP'].unique()

array(['03-06-2024---15:51', '03-06-2024---15:59'], dtype=object)

In [14]:
# Divide il dataframe in base a orari ricerche per fare poi il merge
orario1 = df["TIMESTAMP"].unique()[0]
orario2 = df["TIMESTAMP"].unique()[1]

df1 = df[df["TIMESTAMP"]==orario1]
df2 = df[df["TIMESTAMP"]==orario2]

# Eliminazione duplicati
df1 = df1.drop_duplicates()
df2 = df2.drop_duplicates()

In [19]:
# MERGE TRA DATAFRAME
'''
Se le ricerche sono dello stesso tipo (Mobile o Desktop) il merge avviene anche sulla colonna stanza
Vengono inoltre compattate eventuali colonne uguali e prese in considerazione le righe in cui vi è una differenza di prezzo
'''
df_list = df.columns.tolist()
df_list =  df_list[4:5] +df_list[6:-2]
if df1['user_agent_type'].unique()[0]==df2['user_agent_type'].unique()[0]:
    merged_df = pd.merge(df1, df2, on=['nome_hotel','stanza','data','prezzo_iniziale'], how='inner')
else:
    merged_df = pd.merge(df1, df2, on=['nome_hotel','data'], how='inner')
merged_df = merged_df[merged_df["prezzo_x"]!=merged_df["prezzo_y"]]
for col in df_list:
    if merged_df[col+"_x"].equals(merged_df[col+"_y"]):
        merged_df[col] = merged_df[col+"_x"]
        merged_df.drop(columns=[col+"_x",col+"_y"],inplace=True)
# Elimino hotel con stesso nome
merged_df = merged_df.drop_duplicates(subset="nome_hotel",keep= False)        


In [20]:
# Per visualizzare tutte le righe con differenze di prezzo
#pd.set_option('display.max_rows', None)
merged_df[["nome_hotel","prezzo_x","prezzo_y","offerte_x","offerte_y"]]
len(merged_df)


55

In [23]:
#aggiungo colonna con differenza prezzi
pd.set_option('display.max_rows', None)
merged_df["differenza_prezzo"] = (merged_df["prezzo_x"]-merged_df["prezzo_y"])
merged_df = merged_df.sort_values(by='differenza_prezzo',ascending= False)
#merged_df = merged_df[(merged_df['offerte_x'].isna())&(merged_df['offerte_y'].isna())&(merged_df['stanza_y'].isna())]
merged_df[['nome_hotel','prezzo_x','prezzo_y','differenza_prezzo','offerte_x','offerte_y','stanza_x','stanza_y','TIMESTAMP_x','TIMESTAMP_y']]
merged_df[merged_df['prezzo_iniziale_x']!=merged_df['prezzo_iniziale_y']]



,nome_hotel,prezzo_iniziale_x,prezzo_x,stanza_x,città_x,data,numero_recensioni_x,distanza_centro_x,offerte_x,senza_pagamento_anticipato_x,...,TIMESTAMP_y,offerta_solo_mobile_y,offerta_super_segreta_y,offerta_inizio_2024_y,punteggio,genius,colazione_inclusa,booking_paga,offerta_tempo_limitato,differenza_prezzo
27,Manu's Guest Suite,117,101,Appartamento Deluxe,Cosenza,2024-03-20,108 recensioni,300 m dal centro,NaN,True,...,03-06-2024---15:59,True,False,False,"9,7",True,False,False,False,10
21,Central Home B&B,74,67,Camera Matrimoniale con Bagno Privato,Cosenza,2024-03-20,250 recensioni,450 m dal centro,NaN,False,...,03-06-2024---15:59,True,False,False,"8,6",True,True,False,False,6
43,B&B Tribunale,71,71,Camera Matrimoniale con Terrazza,Cosenza,2024-03-20,77 recensioni,"1,7 km dal centro",NaN,False,...,03-06-2024---15:59,False,False,False,"9,4",False,False,False,False,5


In [ ]:
# Visualizza dove compare una specifica offerta in almeno una delle ricerche
df_mobile = merged_df[(merged_df['booking_paga_x']) ^ (merged_df['booking_paga_y'])]
len(df_mobile)

In [ ]:
# Elimino righe in cui il mobile non ha la stanza
if merged_df['user_agent_type_x'].unique()[0] == 'mobile':
    df_senza_stanza = merged_df['stanza_x'].dropna()
elif merged_df['user_agent_type_y'].unique()[0] == 'mobile':
    df_senza_stanza = merged_df['stanza_y'].dropna()
len(df_senza_stanza)